In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

2023-05-21 14:52:13.476905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-21 14:52:13.476925: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### Goal: Denormalize data

In [ ]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

#### 1. Load Data

In [31]:
dat_train_proc_norm = ldp.load_data_all_train_proc_norm()
dat_test_proc_norm = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

#### 2. Load Means and Stds

1. Variable Index Positions:<br>
    u10: 0 <br>
    v10: 1 <br>
    t2m: 2 <br>
    t850: 5 <br>
    z500: 14 <br>

In [5]:
global_means = np.load("/mnt/sda/Data2/fourcastnet/data/stats_v0/global_means.npy")
global_stds = np.load("/mnt/sda/Data2/fourcastnet/data/stats_v0/global_stds.npy")

<IPython.core.display.Javascript object>

In [9]:
global_means = global_means.flatten()[[0, 1, 2, 5, 14]]

array([-6.42138980e-03,  2.93685094e-01,  2.79020667e+02,  9.66010672e+04,
        1.00906416e+05,  2.74948004e+02,  1.07133299e-02,  3.00078022e-01,
        6.98255676e+02,  1.47735069e+00,  2.15618633e-01,  1.37356820e+04,
        6.35723982e+00, -3.78661333e-02,  5.41704141e+04,  2.02868243e+02,
        1.99242691e+05,  4.99529678e+01,  6.90421692e+01,  1.87243168e+01,
       -1.09157430e+04])

<IPython.core.display.Javascript object>

In [15]:
global_means.flatten()[[0, 1, 2, 5, 14]]

array([-6.42138980e-03,  2.93685094e-01,  2.79020667e+02,  2.74948004e+02,
        5.41704141e+04])

<IPython.core.display.Javascript object>

In [14]:
global_stds.flatten()[[0, 1, 2, 5, 14]]

array([   5.61282367,    4.77990804,   21.42417831,   15.74041515,
       3439.80138962])

<IPython.core.display.Javascript object>